In [1]:
# load in dependent libraries
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk.classify
from collections import defaultdict
import re
from nltk import *
import collections, itertools
from nltk.metrics import precision, recall
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer 
#from featx import bag_of_words, high_information_words

In [2]:
# load in the training data
train_df = pd.read_csv("hcr_omd.csv")
# retrieve the possible labels from the training data
categories = list(set(train_df['sentiment']))
categories

['negative', 'positive', 'neutral']

In [3]:
train_df

,Unnamed: 0,tweet,sentiment
0,0,"Despite reports, Pelosi aides say Rep. Stupak ...",neutral
1,1,Looks like Pelosi is getting ready to play wha...,negative
2,2,It is great to know that there are people who ...,positive
3,3,3/21 the anniversary of my mother's death. Ins...,positive
4,4,Black leaders supporting #hcr bill are more ak...,negative
5,5,Contact @LorettaSanchez CoS & urge YES on #HCR...,positive
6,6,These guys are straight up parroting Glenn Bec...,negative
7,7,RT @Gabe_Holmstrom: If R's were as concerned a...,negative
8,8,RT @MediaLizzy: STUPAK office recording yeas &...,neutral
9,9,STUPAK office recording yeas & nays of callers...,neutral


In [4]:
#train_df = train_df.drop(['Unnamed: 0.1'],1)
#train_df = train_df.reindex(columns=['Unnamed: 0','tweet','sentiment'])
#train_df

In [5]:
# mannaully coppied the featx bag_of_words and high_information_words module since featx package is not available for conda
def high_information_words(labelled_words, score_fn=BigramAssocMeasures.chi_sq, min_score=5):
    word_fd = FreqDist()
    label_word_fd = ConditionalFreqDist()

    for label, words in labelled_words:
        for word in words:
            word_fd[word] += 1
            label_word_fd[label][word] += 1

    n_xx = label_word_fd.N()
    high_info_words = set()

    for label in label_word_fd.conditions():
        n_xi = label_word_fd[label].N()
        word_scores = collections.defaultdict(int)

        for word, n_ii in label_word_fd[label].items():
            n_ix = word_fd[word]
            score = score_fn(n_ii, (n_ix, n_xi), n_xx)
            word_scores[word] = score

        bestwords = [word for word, score in word_scores.items() if score >= min_score]
        high_info_words |= set(bestwords)

    return high_info_words



def bag_of_words(words):
    return dict([(word, True) for word in words])

REMINDER! : CONSIDER REMOVING STOPWORDS, STEMMING

In [6]:
def tokenize(tweet):
    tokens = word_tokenize(tweet)
    tokens = [token.lower() for token in tokens]
    # remove items that are not alphabetics
    for token in tokens:
        if token.isalpha() == False:
            tokens.remove(token)
    # remove punctuation and other tokens
    for token in tokens:
        if token in '!;@#$%^&*().,\/?~1234567890':
            tokens.remove(token)
    
    # remove urls
    regex = re.compile(r'^[/]+')
    for token in tokens:
        if regex.match(token):
            tokens.remove(token)
    
    
    # remove stopwords
    stop_words = set(stopwords.words('english')) 
    for token in tokens: 
        if token in stop_words: 
            tokens.remove(token) 
          
    # word stemmer
    stemmed_tokens = []
    ps = PorterStemmer() 
    for token in tokens:
        stemmed = ps.stem(token)
        stemmed_tokens.append(stemmed)
        
    # Snowbal-stemmer (WORKS BETTER)    
    from nltk.stem.snowball import SnowballStemmer
    stemmed_toks = []
    stemmer = SnowballStemmer("english")
    for token in tokens:
        stemmed = stemmer.stem(token)
        stemmed_toks.append(stemmed)
    
    bag = bag_of_words(stemmed_toks)
    return bag
    

In [7]:
train_df.head()

,Unnamed: 0,tweet,sentiment
0,0,"Despite reports, Pelosi aides say Rep. Stupak ...",neutral
1,1,Looks like Pelosi is getting ready to play wha...,negative
2,2,It is great to know that there are people who ...,positive
3,3,3/21 the anniversary of my mother's death. Ins...,positive
4,4,Black leaders supporting #hcr bill are more ak...,negative


In [8]:
# create list of training features
train_feats = list ()
counter = 0
for i, row in train_df.iterrows():
    if row[2] in categories:    
        bag_cat = row[2]
    tweet_text = row[1]
    counter += 1
    # tokenize tweets and put tokens in bag of words
    bag = tokenize(tweet_text)
    train_feats.append((bag, bag_cat))
    if counter % 100000 == 0:
        print(counter)


In [9]:
# obtain the high information words
def high_information(train_feats, categories):
    labelled_words = [(category, []) for category in categories]

    # convert the formatting of our features to that required by high_information_words
    words = defaultdict(list)
    all_words = list()
    for category in categories:
        words[category] = list()

    for feat in train_feats:
        category = feat[1]
        bag = feat[0]
        for w in bag.keys():
            words[category].append(w)
            all_words.append(w)
            #break

    labelled_words = [(category, words[category]) for category in categories]
    
    # calculate high information words
    # note: to adjust min occurence score add min_score = n
    high_info_words = set(high_information_words(labelled_words,min_score = 2))
    #print(high_info_words)
    #high_info_words contains a list of high-information words. You may want to use only these for classification.
    # You can restrict the words in a bag of words to be in a given 2nd list (e.g. in function read_files)
    # e.g. bag_of_words_in_set(words, high_info_words)
    
    print("  Number of words in the data: %i" % len(all_words))
    print("  Number of distinct words in the data: %i" % len(set(all_words)))
    print("  Number of distinct 'high-information' words in the data: %i" % len(high_info_words))

    return high_info_words


In [10]:
high_info_words = high_information(train_feats, categories)

  Number of words in the data: 50286
  Number of distinct words in the data: 6612
  Number of distinct 'high-information' words in the data: 2839


In [11]:
high_info_words = high_information(train_feats, categories)
new_feats = []
for tup in train_feats:
    dic, sentiment = tup
    new_dict = dict((k,v) for k,v in dic.items() if k in high_info_words)
    new_feats.append(tuple((new_dict, sentiment)))

train_feats = new_feats

  Number of words in the data: 50286
  Number of distinct words in the data: 6612
  Number of distinct 'high-information' words in the data: 2839


In [12]:
# trains a classifier
def train(train_feats):
    classifier = nltk.classify.NaiveBayesClassifier.train(train_feats)
    return classifier
    # the following code uses the classifier with add-1 smoothing (Laplace)
    # You may choose to use that instead
    #from nltk.probability import LaplaceProbDist
    #classifier = nltk.classify.NaiveBayesClassifier.train(train_feats, estimator=LaplaceProbDist)


In [13]:
train_feats[0]

({'despit': True,
  'aid': True,
  'say': True,
  'stupak': True,
  'not': True,
  'yes': True,
  'vote': True,
  'yet': True,
  'hcr': True},
 'neutral')

In [14]:
classifier = train(train_feats)

In [15]:
classifier

In [16]:
# create list of development features
tweet = 'have a bad nice good day' 
# tokenize tweets and put tokens in bag of words
bag = tokenize(tweet)
print(classifier.classify(bag))

negative


In [17]:
# load in the development data
dev_df = pd.read_csv("hcr_test_data.csv")
# retrieve the possible labels from the training data
dev_df.head()

,Unnamed: 0,tweet id,content,sentiment
0,0,10729879540,50% of FoxNews.com readers think #hcr won't pa...,negative
1,1,10740632762,NEW #teaparty sign download ---- Taking back o...,positive
2,2,10780349787,RT @Marnus3: Note to Teabaggers: We took back ...,positive
3,3,10781009520,RT @hippieprof: RT @loudhearted: RT @quaigee: ...,negative
4,4,10727641557,I'm confident those that support #hcr will be ...,negative


In [18]:
tweet = dev_df.loc[3,'content']
classifier.classify(tokenize(tweet))

'negative'

In [19]:

#type(classifier.classify(
#    tokenize(
#        dev_df.content)))
#dev_df['predicted'] = classifier.classify(tokenize(dev_df['content']))
#dev_df.head()
predictions = []
for i, row in dev_df.iterrows():
    tweet = row[2]
    prediction = classifier.classify(tokenize(tweet))
    predictions.append(prediction)
                                     
dev_df['predicted'] = predictions
dev_df

,Unnamed: 0,tweet id,content,sentiment,predicted
0,0,10729879540,50% of FoxNews.com readers think #hcr won't pa...,negative,negative
1,1,10740632762,NEW #teaparty sign download ---- Taking back o...,positive,negative
2,2,10780349787,RT @Marnus3: Note to Teabaggers: We took back ...,positive,negative
3,3,10781009520,RT @hippieprof: RT @loudhearted: RT @quaigee: ...,negative,negative
4,4,10727641557,I'm confident those that support #hcr will be ...,negative,positive
5,5,10781735057,RT @BrettR4763: RT @Marnus3: Rep. Weiner expos...,negative,negative
6,6,10723752009,@margaretacker #hcr I will not be muzzled! I'm...,negative,negative
7,7,10726487327,GO TO YOUR US REPS OFFICE ON SATURDAY AND SAY ...,negative,positive
8,8,10739614533,Retweet this if you are against a government t...,negative,negative
9,9,10739944453,RT @RepKevinBrady: Retweet this if you are aga...,negative,negative


In [20]:
true_positive = 0
#true_negative = 0
false_positive = 0
#false_negative = 0


for i, row in dev_df.iterrows():
    sent = row[3]
    pred = row[4]
    if sent == pred:
        true_positive += 1
    elif sent != pred:
        false_positive += 1
print("TRUE",true_positive)
print("FALSE",false_positive)

TRUE 521
FALSE 277


In [21]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score

precision_recall_fscore_support(dev_df.sentiment, dev_df.predicted, labels = ['negative','neutral','positive'])#(, average='micro')
accuracy_score(dev_df.sentiment, dev_df.predicted)


0.6528822055137845

In [6]:
# load in the development data
dev_df = pd.read_csv("data/HCR/dev/orig/HCR_formatted_dev.csv")
# retrieve the possible labels from the training data
categories = list(set(dev_df['sentiment']))
categories

['neutral', 'positive', 'negative']

In [16]:
# create list of development features
dev_feats = list ()
for i, row in dev_df.iterrows():
    if row[3] in categories:
        bag_cat = row[3]
    tweet_text = row[2]
    # tokenize tweets and put tokens in bag of words
    bag = tokenize(tweet_text)
    dev_feats.append((bag, bag_cat))
bag

{'every': True,
 'time': True,
 'boehner': True,
 'crys': True,
 'an': True,
 'angel': True,
 'gets': True,
 'it': True,
 'wings': True,
 'hcr': True}

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\12859087\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [22]:
def evaluation(classifier, test_feats, categories):
    print ("\n##### Evaluation...")
    print("  Accuracy: %f" % nltk.classify.accuracy(classifier, test_feats))
    precisions, recalls = precision_recall(classifier, test_feats)
    print(precisions, recalls)
    f_measures = calculate_f(precisions, recalls)  

    print(" |-----------|-----------|-----------|-----------------|")
    print(" |%-11s|%-11s|%-11s|%-11s|" % ("category","precision","recall","F-measure"))
    print(" |-----------|-----------|-----------|-----------------|")
    for category in categories:
        if precisions[category] is None:
            print(" |%-11s|%-11s|%-11s|%-11s|" % (category, "NA", "NA", "NA"))
        else:
            print(" |%-11s|%-11f|%-11f|%-11s|" % (category, precisions[category], recalls[category], f_measures[category]))
    print(" |-----------|-----------|-----------|------------------|")

In [23]:
from classification import precision_recall

In [24]:
def calculate_f(precisions, recalls):
    f_measures = {}
    #TODO calculate the f measure for each category using as input the precisions and recalls
    for precision, recall in zip(precisions.items(), recalls.items()):
        f_measures[precision[0]] = ((2*precision[1])*recall[1])/ (precision[1] + recall[1])
    return f_measures

In [25]:
def precision_recall(classifier, testfeats):
    refsets = collections.defaultdict(set)
    testsets = collections.defaultdict(set)
    
    for i, (feats, label) in enumerate(testfeats):
        refsets[label].add(i)
        observed = classifier.classify(feats)
        testsets[observed].add(i)
    
    precisions = {}
    recalls = {}
    
    for label in classifier.labels():
        precisions[label] = precision(refsets[label], testsets[label])
        recalls[label] = recall(refsets[label], testsets[label])
    
    return precisions, recalls

In [26]:
evaluation(classifier, dev_feats, categories)


##### Evaluation...
  Accuracy: 1.000000
{'negative': 1.0, 'neutral': None, 'positive': None} {'negative': 1.0, 'neutral': None, 'positive': None}


TypeError: unsupported operand type(s) for *: 'int' and 'NoneType'